In [1]:
#Cell to create a sparksession

from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 PROJECT 1 DUSTIN")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

22/08/05 17:00:08 WARN Utils: Your hostname, DESKTOP-3ADPNV0 resolves to a loopback address: 127.0.1.1; using 172.23.75.175 instead (on interface eth0)
22/08/05 17:00:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/05 17:00:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import seaborn as sns
from matplotlib import pyplot as pl

In [3]:
sdf_yellow = spark.read.parquet('../../mast30034-project-1-dustintano10/data/raw/yellow/')
sdf_yellow.show(10, vertical=True, truncate=100)

-RECORD 0------------------------------------
 VendorID              | 1                   
 tpep_pickup_datetime  | 2019-01-01 11:46:40 
 tpep_dropoff_datetime | 2019-01-01 11:53:20 
 passenger_count       | 1.0                 
 trip_distance         | 1.5                 
 RatecodeID            | 1.0                 
 store_and_fwd_flag    | N                   
 PULocationID          | 151                 
 DOLocationID          | 239                 
 payment_type          | 1                   
 fare_amount           | 7.0                 
 extra                 | 0.5                 
 mta_tax               | 0.5                 
 tip_amount            | 1.65                
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 9.95                
 congestion_surcharge  | null                
 airport_fee           | null                
-RECORD 1------------------------------------
 VendorID              | 1        

In [4]:
from pyspark.sql import functions as F
sdf_yellow.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)



In [7]:
#filters out the records with 2018 as its year for the yellow taxi dataset
#somehow there are 2018 records in the dataset even though it is a 2019 dataset

sdf_yellow = sdf_yellow.filter(F.col('tpep_pickup_datetime') >= F.lit('2019-01-01'))

#filters out the payment_types that are not credit card or cash as they are irrelevant
sdf_yellow = sdf_yellow.where(
             (F.col('payment_type') == '1')|
             (F.col('payment_type') == '2')
            )


#change the payment_type into a string as it just acts as a categorical attribute
sdf_yellow = sdf_yellow.withColumn(
    'payment_type',
    (F.col("payment_type")).cast('string')
)
sdf_yellow.printSchema()

sdf_yellow.groupby('payment_type').count()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)



payment_type,count
1,32186683
2,11966017


In [7]:
sf = gpd.read_file("../../data/taxi_zones/taxi_zones.shp")
zones = pd.read_csv("../../data/taxi_zones/taxi+_zone_lookup.csv")

sf.head()

22/08/05 16:50:06 ERROR Executor: Exception in task 0.0 in stage 5.0 (TID 15)
java.lang.OutOfMemoryError: Java heap space
	at java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:57)
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:335)
	at org.apache.parquet.bytes.HeapByteBufferAllocator.allocate(HeapByteBufferAllocator.java:32)
	at org.apache.parquet.hadoop.ParquetFileReader$ConsecutivePartList.readAll(ParquetFileReader.java:1700)
	at org.apache.parquet.hadoop.ParquetFileReader.readNextRowGroup(ParquetFileReader.java:925)
	at org.apache.parquet.hadoop.ParquetFileReader.readNextFilteredRowGroup(ParquetFileReader.java:956)
	at org.apache.spark.sql.execution.datasources.parquet.SpecificParquetRecordReaderBase$ParquetRowGroupReaderImpl.readNextRowGroup(SpecificParquetRecordReaderBase.java:266)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedParquetRecordReader.checkEndOfRowGroup(VectorizedParquetRecordReader.java:388)
	at org.apache.spark.sql.execution.datasources.parquet.

22/08/05 16:50:07 ERROR Inbox: Ignoring error
java.util.concurrent.RejectedExecutionException: Task org.apache.spark.executor.Executor$TaskRunner@73a96a49 rejected from java.util.concurrent.ThreadPoolExecutor@2c4e3b95[Shutting down, pool size = 8, active threads = 8, queued tasks = 0, completed tasks = 15]
	at java.util.concurrent.ThreadPoolExecutor$AbortPolicy.rejectedExecution(ThreadPoolExecutor.java:2063)
	at java.util.concurrent.ThreadPoolExecutor.reject(ThreadPoolExecutor.java:830)
	at java.util.concurrent.ThreadPoolExecutor.execute(ThreadPoolExecutor.java:1379)
	at org.apache.spark.executor.Executor.launchTask(Executor.scala:305)
	at org.apache.spark.scheduler.local.LocalEndpoint.$anonfun$reviveOffers$1(LocalSchedulerBackend.scala:93)
	at org.apache.spark.scheduler.local.LocalEndpoint.$anonfun$reviveOffers$1$adapted(LocalSchedulerBackend.scala:91)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collect

ERROR:root:Exception while sending command.                        (0 + 5) / 12]
Traceback (most recent call last):
  File "/mnt/c/Users/User/notebook/jupyterenv/lib/python3.8/site-packages/pyspark/sql/utils.py", line 190, in deco
    return f(*a, **kw)
  File "/mnt/c/Users/User/notebook/jupyterenv/lib/python3.8/site-packages/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/mnt/c/Users/User/notebook/jupyterenv/lib/python3.8/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/mnt/c/Users/User/notebook/jupyterenv/lib/python3.8/site-packages/py4

ConnectionRefusedError: [Errno 111] Connection refused